In [10]:
# Sel 1: Impor Pustaka dan Konfigurasi Awal

import os
import numpy as np
import pandas as pd # Tambahkan ini jika Anda ingin melihat DataFrame secara langsung di notebook
import joblib
from tensorflow.keras.models import load_model # Meskipun train_autoencoder mengembalikan model, ini untuk konsistensi
import time

# Impor fungsi-fungsi yang diperlukan dari models.py
# Pastikan models.py ada di direktori yang sama atau di PYTHONPATH
try:
    from models import (
        parse_log_file, 
        preprocess_data, 
        train_autoencoder, 
        train_ocsvm
    )
    print("[INFO] Modul 'models.py' berhasil diimpor.")
except ImportError:
    print("[ERROR] Gagal mengimpor 'models.py'. Pastikan file tersebut ada di direktori yang sama.")
    # Anda mungkin perlu menghentikan eksekusi di sini jika impor gagal
    raise 

print("="*50)
print("       PELATIHAN MODEL DETEKSI ANOMALI (JUPYTER NOTEBOOK)       ")
print("="*50)

# --- 1. Konfigurasi Path ---
# !!! GANTI INI DENGAN PATH KE FILE LOG TRAINING NORMAL ANDA !!!
TRAINING_LOG_FILE_PATH = 'data/hasil_log_jan.txt' # Contoh jika ada di folder 'data'

# Direktori untuk menyimpan model dan artefak
BASE_DIR = os.path.abspath('.') # Menggunakan direktori kerja notebook saat ini
BASE_MODEL_DIR = os.path.join(BASE_DIR, "trained_models_artifacts")
os.makedirs(BASE_MODEL_DIR, exist_ok=True) 

# Path output
AUTOENCODER_MODEL_PATH = os.path.join(BASE_MODEL_DIR, "trained_autoencoder_model.h5")
OCSVM_MODEL_PATH = os.path.join(BASE_MODEL_DIR, "trained_ocsvm_model.joblib")
SCALER_PATH = os.path.join(BASE_MODEL_DIR, "trained_scaler.joblib")
LABEL_ENCODERS_PATH = os.path.join(BASE_MODEL_DIR, "trained_label_encoders.joblib")
TRAINING_MSE_AE_PATH = os.path.join(BASE_MODEL_DIR, "training_mse_ae.npy")

print(f"\n[INFO] Data training akan dibaca dari: {os.path.abspath(TRAINING_LOG_FILE_PATH)}")
print(f"[INFO] Model & Artefak akan disimpan di: {BASE_MODEL_DIR}\n")

# --- 2. Cek File Training ---
if not os.path.exists(TRAINING_LOG_FILE_PATH):
    print(f"[ERROR] File training log '{TRAINING_LOG_FILE_PATH}' tidak ditemukan!")
    print("[ERROR] Silakan perbarui variabel 'TRAINING_LOG_FILE_PATH' di sel ini dan coba lagi.")
else:
    print(f"[INFO] File training '{TRAINING_LOG_FILE_PATH}' ditemukan.")



[INFO] Modul 'models.py' berhasil diimpor.
       PELATIHAN MODEL DETEKSI ANOMALI (JUPYTER NOTEBOOK)       

[INFO] Data training akan dibaca dari: C:\Users\RYNO-PC\Skripsi\data\hasil_log_jan.txt
[INFO] Model & Artefak akan disimpan di: C:\Users\RYNO-PC\Skripsi\trained_models_artifacts

[INFO] File training 'data/hasil_log_jan.txt' ditemukan.


In [11]:
# Sel 2: Parsing & Pra-pemrosesan Data

if os.path.exists(TRAINING_LOG_FILE_PATH): # Lanjutkan hanya jika file ada
    print("[LANGKAH 1/5] Memulai parsing dan pra-pemrosesan data training...")
    start_time = time.time()
    df_train_raw = parse_log_file(TRAINING_LOG_FILE_PATH)

    if df_train_raw.empty:
        print("[ERROR] Parsing data gagal atau file log kosong.")
    else:
        print(f"[INFO] Parsing selesai. Jumlah record mentah: {len(df_train_raw)}")
        print("[INFO] Menampilkan beberapa baris data mentah (head):")
        display(df_train_raw.head()) # Gunakan display() di Jupyter untuk output yang lebih baik

        print("\n[INFO] Melakukan pra-pemrosesan (normalisasi, encoding, dll.)...")
        # Pastikan fungsi preprocess_data dari models.py dipanggil dengan benar
        df_train_scaled, scaler, label_encoders, feature_cols, df_original_for_output_train = preprocess_data(
            df_train_raw.copy(), 
            is_training=True
        )

        if df_train_scaled is None or df_train_scaled.empty:
            print("[ERROR] Pra-pemrosesan data gagal.")
        else:
            end_time = time.time()
            print(f"[SUKSES] Pra-pemrosesan data selesai. Shape data ter-scaled: {df_train_scaled.shape}. Waktu: {end_time - start_time:.2f} detik.")
            print("[INFO] Menampilkan beberapa baris data yang sudah diproses dan di-scaled (head):")
            display(df_train_scaled.head())
            print(f"[INFO] Kolom fitur yang digunakan: {feature_cols}")
else:
    print("[SKIP] Langkah parsing dan pra-pemrosesan dilewati karena file training tidak ditemukan.")



[LANGKAH 1/5] Memulai parsing dan pra-pemrosesan data training...


KeyboardInterrupt: 

In [7]:
# Sel 3: Simpan Scaler & Label Encoders

if 'scaler' in locals() and 'label_encoders' in locals() and scaler and label_encoders: # Cek apakah variabel ada
    print("\n[LANGKAH 2/5] Menyimpan Scaler dan Label Encoders...")
    try:
        joblib.dump(scaler, SCALER_PATH)
        joblib.dump(label_encoders, LABEL_ENCODERS_PATH)
        print(f"[SUKSES] Scaler disimpan di: {SCALER_PATH}")
        print(f"[SUKSES] Label Encoders disimpan di: {LABEL_ENCODERS_PATH}")
    except Exception as e:
        print(f"[ERROR] Gagal menyimpan scaler/encoders: {e}")
else:
    print("[SKIP] Langkah penyimpanan scaler/encoders dilewati karena variabel tidak ditemukan (mungkin error di pra-pemrosesan).")




[LANGKAH 2/5] Menyimpan Scaler dan Label Encoders...
[SUKSES] Scaler disimpan di: C:\Users\RYNO-PC\Skripsi\trained_models_artifacts\trained_scaler.joblib
[SUKSES] Label Encoders disimpan di: C:\Users\RYNO-PC\Skripsi\trained_models_artifacts\trained_label_encoders.joblib


In [8]:
# Sel 4: Latih & Simpan Autoencoder, Hitung & Simpan MSE Training

if 'df_train_scaled' in locals() and not df_train_scaled.empty: # Cek apakah data training ada
    print("\n[LANGKAH 3/5] Melatih model Autoencoder...")
    start_time_ae = time.time()
    input_dim = df_train_scaled.shape[1]
    
    # Anda bisa menyesuaikan epochs di sini
    autoencoder_model, history_ae = train_autoencoder(
        df_train_scaled, 
        input_dim=input_dim, 
        model_save_path=AUTOENCODER_MODEL_PATH, 
        epochs=50 # Contoh epochs, bisa disesuaikan
    )

    if autoencoder_model:
        end_time_ae = time.time()
        print(f"[SUKSES] Model Autoencoder dilatih dan disimpan. Waktu: {end_time_ae - start_time_ae:.2f} detik.")
        
        # Plot history jika mau (membutuhkan matplotlib)
        # import matplotlib.pyplot as plt
        # plt.plot(history_ae.history['loss'], label='Training Loss')
        # plt.plot(history_ae.history['val_loss'], label='Validation Loss')
        # plt.title('Autoencoder Model Loss')
        # plt.ylabel('Loss (MSE)')
        # plt.xlabel('Epoch')
        # plt.legend()
        # plt.show()

        print("\n[LANGKAH 4/5] Menghitung & Menyimpan MSE Training Autoencoder...")
        try:
            train_predictions_ae = autoencoder_model.predict(df_train_scaled)
            # Pastikan df_train_scaled adalah numpy array untuk operasi pengurangan jika perlu
            df_train_scaled_np = df_train_scaled.to_numpy() if isinstance(df_train_scaled, pd.DataFrame) else df_train_scaled
            training_mse_ae = np.mean(np.power(df_train_scaled_np - train_predictions_ae, 2), axis=1)
            
            np.save(TRAINING_MSE_AE_PATH, training_mse_ae)
            print(f"[SUKSES] Training MSE untuk Autoencoder disimpan di: {TRAINING_MSE_AE_PATH}")
            print(f"[INFO] Statistik MSE Training: Min={training_mse_ae.min():.6f}, Max={training_mse_ae.max():.6f}, Mean={training_mse_ae.mean():.6f}, Median={np.median(training_mse_ae):.6f}")
        except Exception as e:
            print(f"[ERROR] Gagal menghitung atau menyimpan MSE training: {e}")
    else:
        print("[ERROR] Pelatihan Autoencoder gagal.")
else:
    print("[SKIP] Langkah pelatihan Autoencoder dilewati karena data training (df_train_scaled) tidak tersedia.")




[LANGKAH 3/5] Melatih model Autoencoder...
Melatih Autoencoder dengan 50 epochs...
Epoch 1/50
43/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1430 

62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1246 - val_loss: 0.0340
Epoch 2/50
44/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0331 

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0316 - val_loss: 0.0121
Epoch 3/50
46/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0168 

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0163 - val_loss: 0.0072
Epoch 4/50
45/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0102 

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0098 - val_loss: 0.0021
Epoch 5/50
43/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0067 

...

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - val_loss: 2.8567e-04
Epoch 24/50
42/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0011     

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - val_loss: 2.6027e-04
Epoch 25/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - val_loss: 2.8603e-04
Epoch 26/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - val_loss: 3.0245e-04
Epoch 27/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0011 - val_loss: 2.9394e-04
Epoch 28/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.4133e-04 - val_loss: 2.7445e-04
Epoch 29/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.1995e-04 - val_loss: 2.9128e-04
Epoch 30/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.7506e-04 - val_loss: 2.6228e-04
Epoch 31/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4138e-04 - val_loss: 2.8485e-04
Epoch 32/50
41/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.6999e-04 

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.6319e-04 - val_loss: 2.5786e-04
Epoch 33/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.9132e-04 - val_loss: 2.9029e-04
Epoch 34/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.8518e-04 - val_loss: 3.8229e-04
Epoch 35/50
40/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.1010e-04 

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1328e-04 - val_loss: 2.3995e-04
Epoch 36/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.1290e-04 - val_loss: 2.8031e-04
Epoch 37/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.8924e-04 - val_loss: 2.6349e-04
Epoch 38/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7692e-04 - val_loss: 2.8359e-04
Epoch 39/50
42/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.4292e-04 

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.4363e-04 - val_loss: 2.2125e-04
Epoch 40/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.0917e-04 - val_loss: 3.3065e-04
Epoch 41/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.2102e-04 - val_loss: 2.3180e-04
Epoch 42/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.5232e-04 - val_loss: 2.3642e-04
Epoch 43/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.7539e-04 - val_loss: 2.3939e-04
Epoch 44/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1835e-04 - val_loss: 2.3452e-04
Epoch 45/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.9188e-04 - val_loss: 2.6941e-04
Epoch 46/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1266e-04 - val_loss: 2.7061e-04
Epoch 47/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.6638e-04 - val_loss: 2.4267e-04
Epoch 48/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.2621e-04 - val_loss: 2.5460e-04
Epoch 49/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1723e-04 - val_loss: 

In [9]:
# Sel 5: Latih & Simpan One-Class SVM

if 'df_train_scaled' in locals() and not df_train_scaled.empty: # Cek apakah data training ada
    print("\n[LANGKAH 5/5] Melatih model One-Class SVM...")
    start_time_ocsvm = time.time()
    
    ocsvm_model = train_ocsvm(
        df_train_scaled, 
        model_save_path=OCSVM_MODEL_PATH
    )

    if ocsvm_model:
        end_time_ocsvm = time.time()
        print(f"[SUKSES] Model One-Class SVM dilatih dan disimpan. Waktu: {end_time_ocsvm - start_time_ocsvm:.2f} detik.")
        
        # Anda bisa mencoba memprediksi beberapa sampel dari data training untuk melihat hasilnya
        # if len(df_train_scaled) > 0:
        #     predictions_ocsvm_sample = ocsvm_model.predict(df_train_scaled.head())
        #     decision_scores_sample = ocsvm_model.decision_function(df_train_scaled.head())
        #     print("\n[INFO] Contoh prediksi OC-SVM pada data training (head):")
        #     print(f"Label (-1 anomali, 1 normal): {predictions_ocsvm_sample}")
        #     print(f"Skor Keputusan: {decision_scores_sample}")
            
    else:
        print("[ERROR] Pelatihan One-Class SVM gagal.")
        
    print("\n" + "="*50)
    print("     PELATIHAN MODEL SELESAI!     ")
    print("="*50)
    print("Periksa folder 'trained_models_artifacts' untuk file-file yang dihasilkan.")

else:
    print("[SKIP] Langkah pelatihan One-Class SVM dilewati karena data training (df_train_scaled) tidak tersedia.")




[LANGKAH 5/5] Melatih model One-Class SVM...
Melatih OC-SVM...
OC-SVM dilatih dan disimpan di C:\Users\RYNO-PC\Skripsi\trained_models_artifacts\trained_ocsvm_model.joblib
[SUKSES] Model One-Class SVM dilatih dan disimpan. Waktu: 0.02 detik.

     PELATIHAN MODEL SELESAI!     
Periksa folder 'trained_models_artifacts' untuk file-file yang dihasilkan.
